# crewAI Chessmaster

In [1]:
import chess
import chess.engine

In [2]:
from langchain_community.chat_models import ChatOllama
# from langchain_experimental.llms.ollama_functions import OllamaFunctions

llm = ChatOllama(model="llama3.1", temperature=0.1)
# llm = OllamaFunctions(model="llama3.1", format="json")

In [3]:
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task

# Uncomment the following line to use an example of a custom tool
# from chess.tools.custom_tool import MyCustomTool

# Check our tools documentations for more information on how to use them
# from crewai_tools import SerperDevTool

chess_master = Agent(
			role='Chess Grandmaster',
    		goal='Predict best next move to win the chess game',
    		backstory="""As one of the greatest chess players of all time, 
				Known for your deep strategic understanding, innovative approaches, and fierce competitive spirit.""",
			# tools=[MyCustomTool()], # Example of custom tool, loaded on the beginning of file
			llm=llm,
			verbose=True
			)

next_move = Task(
			description="""Conduct a thorough analysis of board state {board} and moves history so far {moves}
    			Predict the best next move
    			Here is the list of legal move:{legal_moves}\n
    			Here is the feedback {feedback}""",
			expected_output='The best next move. This best move can be anyone in the legal move list not just the first one. And only output the move, nothing else.',
			agent=chess_master
			)

chess_crew = Crew(
			agents=[chess_master], # Automatically created by XXX @agent decorator
			tasks=[next_move], # Automatically created by XXX @task decorator
			process=Process.sequential,
			verbose=2,
			# process=Process.hierarchical, # In case you wanna use that instead XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
		)

In [4]:
class ChessGame:
    # players can be llm, engine, user
    def __init__(self, ai_player='llm', board=None, engine_config=None):
        self.ai_player = ai_player
        if engine_config is not None:
            # engine_config = {"path": "/opt/homebrew/Cellar/stockfish/16/bin/stockfish", "skill_lv": 0}
            # Initialize the chess engine
            self.engine = chess.engine.SimpleEngine.popen_uci(engine_config['path'])
            # Configure Stockfish to a specific skill level
            self.engine.configure({"Skill Level": 0})

        # Initialize a chess board
        self.board = board or chess.Board()
        self.moves = []

    def get_agent_move(self):
        feedback = ""
        board = self.board
        while True:
            # Replace with your inputs, it will automatically interpolate any tasks and agents information
            inputs = {
                'board': board.fen(),
                'moves': self.moves,
                'legal_moves': ",".join(
                    [str(_move) for _move in board.legal_moves]),
                'feedback': feedback,
            }

            _result = chess_crew.kickoff(inputs=inputs)
            next_move = _result

            try:
                _move = chess.Move.from_uci(next_move)

                if _move in board.legal_moves:
                    return _move
                else:
                    feedback = f"Agent's generated move {_move} is not valid currently."
            except Exception as e:
                feedback = "Failed to parse the Agent's generated move. Retrying..."

    # def get_user_move(self):
    #     user_move = input("Possible moves are: " + ",".join(
    #         [str(_move) for _move in self.board.legal_moves]
    #     ) + ". Please make your move")
    #     return chess.Move.from_uci(user_move)

    def next(self, user_input=None):
        board = self.board
        move = None
        if user_input:
            try:
                move = chess.Move.from_uci(user_input)
            except:
                print("Invalid move, please try again")
        elif self.ai_player == 'llm':
            move = self.get_agent_move()
        elif self.ai_player == 'engine':
            result = self.engine.play(board, chess.engine.Limit(time=0.001))
            move = result.move
            

        self.board.push(move)
        self.moves.append(move.uci())  # Store UCI move in the list

        return move

    def show_board(self):
        return print(self.board)
    

    def auto_play(self):
        board = self.board
        while not board.is_game_over():
            move = self.next()
            print(f'{"White" if board.turn else "Black"} made a move: {move.uci()}')

        # Check the result of the game
        winner = ""
        if board.is_checkmate():
            if board.turn:
                winner = "Black"
            else:
                winner = "White"
        elif board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition() or board.is_variant_draw():
            winner = "Draw"

        if winner == "Black":
            return "Black wins by checkmate."
        elif winner == "White":
            return "White wins by checkmate."
        else:
            return "The game is a draw."

In [5]:
chess_master = ChessGame()

In [6]:
chess_master.next("e2e3")

Move.from_uci('e2e3')

In [7]:
chess_master.show_board()

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . P . . .
P P P P . P P P
R N B Q K B N R


In [8]:
chess_master.next()

 [2024-08-01 13:16:53][DEBUG]: == Working Agent: Chess Grandmaster
 [2024-08-01 13:16:53][INFO]: == Starting Task: Conduct a thorough analysis of board state rnbqkbnr/pppppppp/8/8/8/4P3/PPPP1PPP/RNBQKBNR b KQkq - 0 1 and moves history so far ['e2e3']
    			Predict the best next move
    			Here is the list of legal move:g8h6,g8f6,b8c6,b8a6,h7h6,g7g6,f7f6,e7e6,d7d6,c7c6,b7b6,a7a6,h7h5,g7g5,f7f5,e7e5,d7d5,c7c5,b7b5,a7a5

    			Here is the feedback 


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer: e7e5

> Finished chain.
 [2024-08-01 13:16:56][DEBUG]: == [Chess Grandmaster] Task output: e7e5




Move.from_uci('e7e5')

In [9]:
chess_master.show_board()

r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . . . . .
. . . . P . . .
P P P P . P P P
R N B Q K B N R
